In [1]:
from mqt.bench import CompilerSettings, QiskitSettings, TKETSettings, get_benchmark
from qiskit import *
import numpy as np

In [2]:
q_num=3 #number of choosen qubit
compiler_settings = CompilerSettings(qiskit=QiskitSettings(optimization_level=1))
qc = get_benchmark(
benchmark_name="qaoa",
level="nativegates",
circuit_size=q_num,
compiler="qiskit",
compiler_settings=compiler_settings,
provider_name="ionq",)
print("---------------qaoa---------------")
print(qc.draw())
print("=================================================================================================")

---------------qaoa---------------
global phase: π/2
        ┌──────────────┐┌──────────────┐ ┌────────────┐               »
   q_0: ┤0             ├┤0             ├─┤ Rz(1.9773) ├───────────────»
        │  Rxx(3.0598) ││              │┌┴────────────┴┐┌────────────┐»
   q_1: ┤1             ├┤  Rxx(3.0598) ├┤0             ├┤ Rz(1.9773) ├»
        └──────────────┘│              ││  Rxx(3.0598) │├────────────┤»
   q_2: ────────────────┤1             ├┤1             ├┤ Rz(1.9773) ├»
                        └──────────────┘└──────────────┘└────────────┘»
meas: 3/══════════════════════════════════════════════════════════════»
                                                                      »
«        ┌────────────┐┌────────────┐┌──────────────┐  ┌─────────┐   »
«   q_0: ┤0           ├┤0           ├┤ Rz(-0.68274) ├──┤ Ry(π/2) ├───»
«        │  Rxx(-2.6) ││            │└┬────────────┬┘┌─┴─────────┴──┐»
«   q_1: ┤1           ├┤  Rxx(-2.6) ├─┤0           ├─┤ Rz(-0.68274) ├»
«        └─────

In [3]:
from qiskit_aer import AerSimulator
from qiskit import *
from qiskit.providers.fake_provider import *
import numpy as np 
import os
from qiskit.primitives import StatevectorSampler
sampler = StatevectorSampler()
job = sampler.run([qc], shots=1000)
result = job.result()
counts= result[0].data["meas"].get_counts()
print(f" > Counts: {result[0].data["meas"].get_counts()}")

 > Counts: {'110': 179, '011': 163, '100': 175, '101': 159, '010': 154, '001': 170}


In [5]:
#print(compilation_information)
def get_ideal_counts(circuit: cirq.Circuit) -> dict[str, float]:
    ideal_counts = {}
    for i, amplitude in enumerate(circuit.final_state_vector(ignore_terminal_measurements=True)):
        bitstring = f"{i:>0{len(circuit.all_qubits())}b}"
        probability = np.abs(amplitude) ** 2
        
        ideal_counts[bitstring] = probability
    return ideal_counts
raw_counts = supermarq.simulation.get_ideal_counts(qaoa.circuit())
ideal_counts = {bitstring[::-1]: probability for bitstring, probability in raw_counts.items()}

NameError: name 'cirq' is not defined

In [ ]:
from qiskit.quantum_info import hellinger_fidelity
ideal_counts = get_ideal_counts(qc)
total_shots = sum(counts.values())
experimental_counts = {k: v / total_shots for k, v in counts.items()}
ideal_value = self._get_expectation_value_from_probs(ideal_counts)
experimental_value = self._get_expectation_value_from_probs(experimental_counts)
return 1 - abs(ideal_value - experimental_value) / (2 * ideal_value)

device_dist = {bitstr: count / total_shots for bitstr, count in counts.items()}
hf = hellinger_fidelity(ideal_dist, device_dist) #formula of Hellinger fidelity between two distributions p and q is given by $(\sum_i{p_i q_i})^2$ as FLOAT.
print("Hellinger fidelity",hf*100,"%")

In [ ]:
from qiskit.visualization import plot_histogram
legend = ['Ideal', 'Current']
plot_histogram([ideal_dist, device_dist], legend=legend, color=['skyblue','lightgreen'],title="Visual Comparision of Counts")

In [ ]:
from __future__ import annotations

import cirq
import numpy as np


def get_ideal_counts(circuit: cirq.Circuit) -> dict[str, float]:
    """Noiseless statevector simulation.

    Note that the qubits in the returned bitstrings are in big-endian order.
    For example, for a circuit defined on qubits
    .. code::

        q0 ------
        q1 ------
        q2 ------

    the bitstrings are written as `q0q1q2`.

    Args:
        circuit: Input `cirq.Circuit` to be simulated.

    Returns:
        A dictionary with bitstring and probability as the key, value pairs.
    """
    ideal_counts = {}
    for i, amplitude in enumerate(circuit.final_state_vector(ignore_terminal_measurements=True)):
        bitstring = f"{i:>0{len(circuit.all_qubits())}b}"
        probability = np.abs(amplitude) ** 2
        ideal_counts[bitstring] = probability
    return ideal_counts